## Importando bibliotecas

In [3]:
import pyodbc
import pandasql as ps
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats

## Conectando ao database

Vamos primeiro conectar ao database que vamos utilizar neste exercicio. Note que, se você utilizar os mesmos nomes que estão em "Server" e "Database" em seu computador, não vai funcionar. Você deve altera-los para seu ambiente especifico.

In [4]:
dados_conexao = (r"Driver={SQL Server};"
                 r"Server=DESKTOP-2TIOC7M\SQLEXPRESS;" 
                 r"Database=Project4Database;"
                 r"Trusted_Connection=yes;")

In [5]:
conexao = pyodbc.connect(dados_conexao)

In [6]:
cursor = conexao.cursor()

## Queries e testes de hipóteses

Vamos agora, com a conexão feita, utilizar  queries semelhantes ao  do exercicio anterior para ver os dados visualmente e testar hipóteses. <br> Antes, vimos o status como "Current" para quem tem asma, "Former" para quem já teve asma  e se "curou", e "never" para que nunca teve asma. Vamos simplificar um pouquinho: vamos só considerar que tem asma atualmente ou não, através da variavel "CASTHM1"( "Adults who have been told they currently have asthma"). <br><br>
Vamos fazer um aquecimento só vendo quais são as correlações entre ter asma e o tipo de locla onde você vive:

### Ter asma/Não ter asma VS Morar em zonas rurais/urbanas

In [7]:
df_Urb_Asth = pd.read_sql("""
SELECT 
URBSTAT,
CASE 
	WHEN URBSTAT = 1  THEN 'Urban'
	WHEN URBSTAT = 2  THEN 'Rural'
	ELSE NULL END AS  URBSTAT_Categories,
CASTHM1,
CASE 
	WHEN CASTHM1  =  1 THEN  'No'
	WHEN CASTHM1  =  2 THEN  'Yes'
	ELSE NULL END AS CASTHM1_Categories,
COUNT(*) AS Count,
ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(PARTITION BY URBSTAT),2) AS Percentage
FROM AsthmaStatus 
INNER JOIN UrbanStatus ON AsthmaStatus.ID = UrbanStatus.ID
WHERE CASTHM1 IS NOT NULL
AND URBSTAT IS NOT NULL
GROUP BY URBSTAT,CASTHM1
ORDER BY CASTHM1 DESC, URBSTAT;""", conexao)
df_Urb_Asth

,URBSTAT,URBSTAT_Categories,CASTHM1,CASTHM1_Categories,Count,Percentage
0,1,Urban,2,Yes,36154,9.87
1,2,Rural,2,Yes,5853,9.49
2,1,Urban,1,No,330273,90.13
3,2,Rural,1,No,55832,90.51


Esta funcionando, agora vamos para a visulização de dados e testes estatísticos para cada hipótese

In [8]:
px.bar(df_Urb_Asth, x="CASTHM1_Categories", y = "Percentage", facet_col= "URBSTAT_Categories" )


Aparentemente, não parecem ser dependentes, porém vamos checar isso com um teste de hipótese.
Vamos simplificar um pouco, vamos tentar ver se tem alguma dependência entre apenas ter asma e morar em um local urbano/rural.

In [9]:
Urb_Asth_contigency = df_Urb_Asth["Count"]
Urb_Asth_contigency = [[Urb_Asth_contigency[0], Urb_Asth_contigency[2]],
                       [Urb_Asth_contigency[1], Urb_Asth_contigency[3]]]
Urb_Asth_contigency = pd.DataFrame(Urb_Asth_contigency, columns=["Com asma", "Sem Asma"], index = ["Vivem em ambientes urbanos", "Vivem em ambientes rurais"])
Urb_Asth_contigency

,Com asma,Sem Asma
Vivem em ambientes urbanos,36154,330273
Vivem em ambientes rurais,5853,55832


In [10]:
stats.chi2_contingency(pd.DataFrame(Urb_Asth_contigency)
)

Chi2ContingencyResult(statistic=8.486600112886542, pvalue=0.003577717710792795, dof=1, expected_freq=array([[ 35954.37406333, 330472.62593667],
       [  6052.62593667,  55632.37406333]]))

Diferente do que haviamos proposto aparentemente, o teste de hipótese aponta que são sim dependentes, porém deve-se olhar para os resultados com devida cautela, dado que normalmente em amostras muito grandes, é comum os valores de p serem menores. Portanto, vamos ver a magnitude dessa dependência através de um Odds ratio.

In [11]:
stats.contingency.odds_ratio(pd.DataFrame(Urb_Asth_contigency))


OddsRatioResult(statistic=1.0442101256289011)

Como pode-se observar a existe 1.04 mais chances de você ter asma se você vive em um ambiente urbano do que em um ambiente rural, o que, na prática, é praticamente insignificante.

### Ter asma/Não ter asma VS Morar em zonas Metropolitanas/Não-metropolitanas

In [12]:
df_Met_Asth = pd.read_sql("""
SELECT 
METSTAT,
CASE 
	WHEN METSTAT = 1  THEN 'Metropolitan'
	WHEN METSTAT = 2  THEN 'Non-metropolitan'
	ELSE NULL END AS  METSTAT_Categories,
CASTHM1,
CASE 
	WHEN CASTHM1  =  1 THEN  'No'
	WHEN CASTHM1  =  2 THEN  'Yes'
	ELSE NULL END AS CASTHM1_Categories,
COUNT(*) AS Count,
ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(PARTITION BY METSTAT),2) AS Percentage
FROM AsthmaStatus 
INNER JOIN UrbanStatus ON AsthmaStatus.ID = UrbanStatus.ID
WHERE CASTHM1 IS NOT NULL
AND METSTAT IS NOT NULL
GROUP BY METSTAT,CASTHM1
ORDER BY CASTHM1 DESC, METSTAT;""", conexao)
df_Met_Asth

,METSTAT,METSTAT_Categories,CASTHM1,CASTHM1_Categories,Count,Percentage
0,1,Metropolitan,2,Yes,29363,9.86
1,2,Non-metropolitan,2,Yes,12644,9.71
2,1,Metropolitan,1,No,268524,90.14
3,2,Non-metropolitan,1,No,117581,90.29


In [13]:
px.bar(df_Met_Asth, x="CASTHM1_Categories", y = "Percentage", facet_col= "METSTAT_Categories" )

Aparentemente, não ha dependência como o anterior, mas vamos verificar adequadamente.

###

In [14]:
Met_Asth_contigency = df_Met_Asth["Count"]
Met_Asth_contigency = [[Met_Asth_contigency[0], Met_Asth_contigency[2]],
                       [Met_Asth_contigency[1], Met_Asth_contigency[3]]]
Met_Asth_contigency = pd.DataFrame(Met_Asth_contigency, columns=["Com asma", "Sem Asma"], index = ["Vivem em ambientes metropolitanos", "Vivem em ambientes não-metropolitanos"])
Met_Asth_contigency

,Com asma,Sem Asma
Vivem em ambientes metropolitanos,29363,268524
Vivem em ambientes não-metropolitanos,12644,117581


In [15]:
stats.chi2_contingency(pd.DataFrame(Met_Asth_contigency))

Chi2ContingencyResult(statistic=2.2184382491202173, pvalue=0.13637094699573296, dof=1, expected_freq=array([[ 29229.12510978, 268657.87489022],
       [ 12777.87489022, 117447.12510978]]))

Wow, dessa vez nossa hipótese foi confirmada, não parece que há uma dependência entre morar em um ambiente metropolitano/não-metropolitano e ter asma ou não. Logo, não precisamos calcular o odds ratio, pois provavelmente, o intervalo de confiança em que o mesmo vai pertencer vai incluir o 1, o que indica que não há dependência.

### Qualidade de vida reportada no geral vs local onde a pessoa vive

Bom, como vimos aparentemente há alguma correlação entre ter asma ou não e viver em ambientes urbanos/rurais. Porém, a pergunta de verdade é quais são os fatores que influenciam as pessoas asmáticas a terem uma melhor qualidade de vida? Vamos tentar analisar todos esses fatores, mas agora vamos filtrar apenas as pessoas que tem asma.

Vamos anotando quais fatores parecem ter correlação com a qualidade de vida no geral da pessoa com asma, por enquanto sabemos só que ter asma ou não esta de alguma forma correlacionado com o lugar onde você vive, portanto vamos começar com esse fator.

In [16]:
df_Gen_Urb = pd.read_sql("""
SELECT  URBSTAT,
CASE 
	WHEN URBSTAT = 1  THEN 'Urban'
	WHEN URBSTAT = 2  THEN 'Rural'
	ELSE NULL END AS  URBSTAT_Categories,
RFHLTH,
CASE
	WHEN RFHLTH = 1  THEN 'Good or Better Health'
	WHEN RFHLTH = 2  THEN 'Fair or Poor Health'
	ELSE NULL END AS  RFHLTH_Categories,
COUNT(*) AS Count,
ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(PARTITION BY URBSTAT),2) AS Percentage
FROM UrbanStatus
INNER JOIN HealthStatus 
ON UrbanStatus.ID = HealthStatus.ID
INNER JOIN AsthmaStatus 
ON UrbanStatus.ID = AsthmaStatus.ID
WHERE URBSTAT IS NOT NULL
AND RFHLTH IS NOT NULL
AND CASTHM1  = 2
GROUP BY URBSTAT,RFHLTH
ORDER BY URBSTAT,RFHLTH;""", conexao)
df_Gen_Urb

,URBSTAT,URBSTAT_Categories,RFHLTH,RFHLTH_Categories,Count,Percentage
0,1,Urban,1,Good or Better Health,25072,69.53
1,1,Urban,2,Fair or Poor Health,10988,30.47
2,2,Rural,1,Good or Better Health,3663,62.79
3,2,Rural,2,Fair or Poor Health,2171,37.21


Realmente parece ter alguma diferença, vamos visualizar:

In [17]:
px.bar(df_Gen_Urb, x="RFHLTH_Categories", y = "Percentage", facet_col= "URBSTAT_Categories" )

É, realmente é uma leve diferença, porém é uma diferença! 

Vamos verificar se ela é signficativa e se for, qual a magnitude dessa diferença.

In [18]:
Met_Gen_Urb= df_Gen_Urb["Count"]
Met_Gen_Urb = [[Met_Gen_Urb[0], Met_Gen_Urb[2]],
                       [Met_Gen_Urb[1], Met_Gen_Urb[3]]]
Met_Gen_Urb = pd.DataFrame(Met_Gen_Urb, columns=["Com saude geral reportada boa ou ótima ", " Com uma saúde geral reportada ruim ou péssima"], index = ["Vivem em ambientes urbanos", "Vivem em ambientes rurais"])
Met_Gen_Urb

,Com saude geral reportada boa ou ótima,Com uma saúde geral reportada ruim ou péssima
Vivem em ambientes urbanos,25072,3663
Vivem em ambientes rurais,10988,2171


In [19]:
stats.chi2_contingency(pd.DataFrame(Met_Gen_Urb))

Chi2ContingencyResult(statistic=105.61682972469669, pvalue=8.946463883772956e-25, dof=1, expected_freq=array([[24733.47257364,  4001.52742636],
       [11326.52742636,  1832.47257364]]))

In [20]:
stats.contingency.odds_ratio(pd.DataFrame(Met_Gen_Urb))


OddsRatioResult(statistic=1.3523527270614424)

Apesar de ter dado significativo vemos que a estimativa é que uma pessoa com asma tem 1.2X mais chances de reportar uma saúde boa vivendo em ambientes urbanos do que em ambientes rurais!

Como estamos repetindo os mesmos comandos varias vezes para cirar a tabela de contigência, acho que vale a pena criamos uma função para fazer tudo isso, então vamos la. Considerando que o dataframe se mantenha igual ao que vimos anteriormente, temos a seguinte função:

In [21]:
def Transforma_tabela_contigencia(dataframe, Nome_das_colunas = None, Nome_das_linhas = None):
    dataframe_contigency = dataframe["Count"]
    dataframe_contigency = [[dataframe_contigency[0], dataframe_contigency[2]],
                       [dataframe_contigency[1], dataframe_contigency[3]]]
    dataframe_contigency = pd.DataFrame(dataframe_contigency, columns=Nome_das_colunas , index = Nome_das_linhas )
    return dataframe_contigency

### Qualidade de saúde física reportada a vs local onde a pessoa vive

Dessa vez, estamos lidando com uma variável numérica, o número de dias que uma pessoa reportou não se sentir bem fisicamente, portanto vamos utilizar outros métodos tanto de visualização quando estatísticos

In [22]:

df_Phy_Urb = pd.read_sql("""
SELECT  
URBSTAT,
CASE 
	WHEN URBSTAT = 1  THEN 'Urban'
	WHEN URBSTAT = 2  THEN 'Rural'
	ELSE NULL END AS  URBSTAT_Categories,
ROUND(AVG(CAST(PHYSHLTH AS FLOAT)),2) AS Mean
FROM UrbanStatus
INNER JOIN HealthStatus 
ON UrbanStatus.ID = HealthStatus.ID
INNER JOIN AsthmaStatus
ON AsthmaStatus.ID = UrbanStatus.ID
WHERE URBSTAT IS NOT NULL
AND CASTHM1 = 2
AND PHYSHLTH IS NOT NULL
GROUP BY URBSTAT
ORDER BY URBSTAT""", conexao)
df_Phy_Urb

,URBSTAT,URBSTAT_Categories,Mean
0,1,Urban,6.89
1,2,Rural,8.12


Parece que a média muda um pouco, vamos olhar o dataset inteiro para vermos se temos algumas medidas mais precisas.

In [23]:
df_Phy_Urb = pd.read_sql("""
SELECT  
URBSTAT,
CASE 
	WHEN URBSTAT = 1  THEN 'Urban'
	WHEN URBSTAT = 2  THEN 'Rural'
	ELSE NULL END AS  URBSTAT_Categories,
PHYSHLTH
FROM UrbanStatus
INNER JOIN HealthStatus 
ON UrbanStatus.ID = HealthStatus.ID
INNER JOIN AsthmaStatus
ON AsthmaStatus.ID = UrbanStatus.ID
WHERE URBSTAT IS NOT NULL
AND CASTHM1  = 2
AND PHYSHLTH IS NOT NULL
""", conexao)
df_Phy_Urb

,URBSTAT,URBSTAT_Categories,PHYSHLTH
0,1,Urban,0
1,2,Rural,0
2,1,Urban,30
3,1,Urban,10
4,2,Rural,0
...,...,...,...
40869,2,Rural,0
40870,1,Urban,0
40871,1,Urban,2
40872,1,Urban,0


Para checar se os grupos são diferentes, vamos utilizar um teste de hipóteses ANOVA. Porém, antes de utiliza-lo vamos checar todas suas condições necessárias para o teste:

### Normalidade

Cada amostra foi retirada de uma população normalmente distribuida. Assim, os residuais do teste devem ser normalmente distribuidos. Vamos testar isso de 3 formas::

Primeiro, vamos ajustar a nova aos dados.

In [25]:
Anova_Phy_Urb = stats.f_oneway(df_Phy_Urb.loc[df_Phy_Urb["URBSTAT"] == 1,"PHYSHLTH"] ,df_Phy_Urb.loc[df_Phy_Urb["URBSTAT"] == 2,"PHYSHLTH"])


In [26]:
Anova_Phy_Urb

F_onewayResult(statistic=65.68017705358741, pvalue=5.449662139255904e-16)

In [24]:
px.histogram(df_Phy_Urb, x=  "PHYSHLTH")

In [49]:
px.box(df_Phy_Urb, y="PHYSHLTH", x = "URBSTAT_Categories"  )

In [52]:
stats.kruskal(df_Phy_Urb.loc[df_Phy_Urb["URBSTAT"] == 1,"PHYSHLTH"] ,df_Phy_Urb.loc[df_Phy_Urb["URBSTAT"] == 2,"PHYSHLTH"])

KruskalResult(statistic=51.43996528354143, pvalue=7.382034635334685e-13)